In [1]:
sc

In [6]:
spark

In [35]:
import pyspark.sql.functions as func


In [23]:
collisions = sc.textFile("NYPD_Motor_Vehicle_Collisions.csv", format='csv', header=True, inferSchema=False)
k = sc.textFile("2016.csv.gz", format='csv', header=True, inferSchema=False)
df = sqlContext.createDataFrame

TypeError: textFile() got an unexpected keyword argument 'format'

In [76]:
df = spark.read.load("NYPD_Motor_Vehicle_Collisions.csv", format='csv', header=True, inferSchema=False)
df2 = spark.read.load("2016.csv.gz", format='csv', header=False, inferSchema=False)

apple = df.groupBy("DATE")



In [75]:
df.take(100)

[Row(DATE=u'05/03/2017', TIME=u'9:00', BOROUGH=None, ZIP CODE=None, LATITUDE=u'40.83801', LONGITUDE=u'-73.87329', LOCATION=u'(40.83801, -73.87329)', ON STREET NAME=None, CROSS STREET NAME=None, OFF STREET NAME=None, NUMBER OF PERSONS INJURED=u'2', NUMBER OF PERSONS KILLED=u'0', NUMBER OF PEDESTRIANS INJURED=u'0', NUMBER OF PEDESTRIANS KILLED=u'0', NUMBER OF CYCLIST INJURED=u'0', NUMBER OF CYCLIST KILLED=u'0', NUMBER OF MOTORIST INJURED=u'2', NUMBER OF MOTORIST KILLED=u'0', CONTRIBUTING FACTOR VEHICLE 1=u'Driver Inattention/Distraction', CONTRIBUTING FACTOR VEHICLE 2=u'Unspecified', CONTRIBUTING FACTOR VEHICLE 3=None, CONTRIBUTING FACTOR VEHICLE 4=None, CONTRIBUTING FACTOR VEHICLE 5=None, UNIQUE KEY=u'3662349', VEHICLE TYPE CODE 1=u'PASSENGER VEHICLE', VEHICLE TYPE CODE 2=u'PASSENGER VEHICLE', VEHICLE TYPE CODE 3=None, VEHICLE TYPE CODE 4=None, VEHICLE TYPE CODE 5=None),
 Row(DATE=u'05/03/2017', TIME=u'0:00', BOROUGH=u'BRONX', ZIP CODE=u'10471', LATITUDE=u'40.89571', LONGITUDE=u'-73.897

In [68]:
a = df.take(1)[0]
from dateutil.parser import parse

In [72]:
parse(a.DATE + " " + a.TIME + "-0500")

datetime.datetime(2017, 5, 3, 9, 0, tzinfo=tzoffset(None, -18000))

In [64]:
a.TIME

u'9:00'

In [31]:
df.dtypes

[('DATE', 'string'),
 ('TIME', 'string'),
 ('BOROUGH', 'string'),
 ('ZIP CODE', 'string'),
 ('LATITUDE', 'string'),
 ('LONGITUDE', 'string'),
 ('LOCATION', 'string'),
 ('ON STREET NAME', 'string'),
 ('CROSS STREET NAME', 'string'),
 ('OFF STREET NAME', 'string'),
 ('NUMBER OF PERSONS INJURED', 'string'),
 ('NUMBER OF PERSONS KILLED', 'string'),
 ('NUMBER OF PEDESTRIANS INJURED', 'string'),
 ('NUMBER OF PEDESTRIANS KILLED', 'string'),
 ('NUMBER OF CYCLIST INJURED', 'string'),
 ('NUMBER OF CYCLIST KILLED', 'string'),
 ('NUMBER OF MOTORIST INJURED', 'string'),
 ('NUMBER OF MOTORIST KILLED', 'string'),
 ('CONTRIBUTING FACTOR VEHICLE 1', 'string'),
 ('CONTRIBUTING FACTOR VEHICLE 2', 'string'),
 ('CONTRIBUTING FACTOR VEHICLE 3', 'string'),
 ('CONTRIBUTING FACTOR VEHICLE 4', 'string'),
 ('CONTRIBUTING FACTOR VEHICLE 5', 'string'),
 ('UNIQUE KEY', 'string'),
 ('VEHICLE TYPE CODE 1', 'string'),
 ('VEHICLE TYPE CODE 2', 'string'),
 ('VEHICLE TYPE CODE 3', 'string'),
 ('VEHICLE TYPE CODE 4', 'str

In [30]:
ok = df2.take(1)[1]

IndexError: list index out of range

In [29]:
ok

Row(_c0=u'US1NJGL0001', _c1=u'20160101', _c2=u'PRCP', _c3=u'0', _c4=None, _c5=None, _c6=u'N', _c7=None)

In [22]:
ok.asDict()

{'BOROUGH': None,
 'CONTRIBUTING FACTOR VEHICLE 1': u'Driver Inattention/Distraction',
 'CONTRIBUTING FACTOR VEHICLE 2': u'Unspecified',
 'CONTRIBUTING FACTOR VEHICLE 3': None,
 'CONTRIBUTING FACTOR VEHICLE 4': None,
 'CONTRIBUTING FACTOR VEHICLE 5': None,
 'CROSS STREET NAME': None,
 'DATE': u'05/03/2017',
 'LATITUDE': u'40.83801',
 'LOCATION': u'(40.83801, -73.87329)',
 'LONGITUDE': u'-73.87329',
 'NUMBER OF CYCLIST INJURED': u'0',
 'NUMBER OF CYCLIST KILLED': u'0',
 'NUMBER OF MOTORIST INJURED': u'2',
 'NUMBER OF MOTORIST KILLED': u'0',
 'NUMBER OF PEDESTRIANS INJURED': u'0',
 'NUMBER OF PEDESTRIANS KILLED': u'0',
 'NUMBER OF PERSONS INJURED': u'2',
 'NUMBER OF PERSONS KILLED': u'0',
 'OFF STREET NAME': None,
 'ON STREET NAME': None,
 'TIME': u'9:00',
 'UNIQUE KEY': u'3662349',
 'VEHICLE TYPE CODE 1': u'PASSENGER VEHICLE',
 'VEHICLE TYPE CODE 2': u'PASSENGER VEHICLE',
 'VEHICLE TYPE CODE 3': None,
 'VEHICLE TYPE CODE 4': None,
 'VEHICLE TYPE CODE 5': None,
 'ZIP CODE': None}

In [4]:
from pyspark.sql import Row
import csv

def parseCSV(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        yield Row(tripduration=float(p[2]),
                  starttime=p[3],
                  start_station_time=p[6])
        
rows = sc.textFile('citibike.csv').mapPartitionsWithIndex(parseCSV)
df = sqlContext.createDataFrame(rows)

In [5]:
df.dtypes

[('start_station_time', 'string'),
 ('starttime', 'string'),
 ('tripduration', 'double')]

In [6]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import csv

def parseCSV(idx, part):
    if idx == 0: 
        part.next()
    for p in csv.reader(part):
        yield (int(p[2]), p[3], p[6])

rows = sc.textFile('citibike.csv').mapPartitionsWithIndex(parseCSV)
schema = StructType([StructField('tripduration', IntegerType()), 
                     StructField('starttime', StringType()), 
                     StructField('start_station_name', StringType())])
df = sqlContext.createDataFrame(rows, schema)
df.dtypes

[('tripduration', 'int'),
 ('starttime', 'string'),
 ('start_station_name', 'string')]

In [7]:
import pyspark.sql.functions as sf

df.agg(sf.approx_count_distinct(df.tripduration).alias('c')).show()

+----+
|   c|
+----+
|2537|
+----+



In [8]:
df.agg(sf.approx_count_distinct((df.tripduration/60).cast('int')).alias('c')).show()

+---+
|  c|
+---+
|169|
+---+



In [9]:
df.registerTempTable('citibike')

In [80]:
df.registerTempTable('nypd')

In [10]:
sqlContext.sql('select * from citibike order by tripduration limit 5').collect()

[Row(tripduration=60, starttime=u'2015-02-06 11:38:00+00', start_station_name=u'W 20 St & 8 Ave'),
 Row(tripduration=60, starttime=u'2015-02-05 09:32:00+00', start_station_name=u'Catherine St & Monroe St'),
 Row(tripduration=60, starttime=u'2015-02-06 20:40:00+00', start_station_name=u'E 10 St & Avenue A'),
 Row(tripduration=60, starttime=u'2015-02-07 21:40:00+00', start_station_name=u'E 5 St & Avenue C'),
 Row(tripduration=60, starttime=u'2015-02-02 21:39:00+00', start_station_name=u'W 31 St & 7 Ave')]

In [101]:
for x in range(1,10):
    date = "0{0}/%%/2015".format(x)
    query = 'select * from nypd where DATE LIKE "{0}"'.format(date)
    print(x, sqlContext.sql(query).count())
for x in range(10,13):
    date = "{0}/%%/2015".format(x)
    query = 'select * from nypd where DATE LIKE "{0}"'.format(date)
    print(x, sqlContext.sql(query).count())

(1, 16122)
(2, 15712)
(3, 17948)
(4, 16789)
(5, 19269)
(6, 18818)
(7, 18764)
(8, 18973)
(9, 18510)
(10, 19847)
(11, 18018)
(12, 18867)


In [102]:
for x in range(1,10):
    date = "0{0}/%%/2016".format(x)
    query = 'select * from nypd where DATE LIKE "{0}"'.format(date)
    print(x, sqlContext.sql(query).count())
for x in range(10,13):
    date = "{0}/%%/2016".format(x)
    query = 'select * from nypd where DATE LIKE "{0}"'.format(date)
    print(x, sqlContext.sql(query).count())

(1, 18099)
(2, 15983)
(3, 18519)
(4, 18349)
(5, 20052)
(6, 19435)
(7, 19867)
(8, 19678)
(9, 19491)
(10, 19667)
(11, 19318)
(12, 19219)


In [107]:
date = "03/08/2015"
query = 'select * from nypd where DATE LIKE "{0}" ANIME limit 100'.format(date)
#values = sqlContext.sql(query).count()
#print(x, sqlContext.sql(query).count())

sqlContext.sql(query).collect()

[Row(DATE=u'03/08/2015', TIME=u'0:01', BOROUGH=u'QUEENS', ZIP CODE=u'11435', LATITUDE=u'40.6869122', LONGITUDE=u'-73.7943714', LOCATION=u'(40.6869122, -73.7943714)', ON STREET NAME=u'SUTPHIN BOULEVARD               ', CROSS STREET NAME=u'113 AVENUE                      ', OFF STREET NAME=None, NUMBER OF PERSONS INJURED=u'1', NUMBER OF PERSONS KILLED=u'0', NUMBER OF PEDESTRIANS INJURED=u'1', NUMBER OF PEDESTRIANS KILLED=u'0', NUMBER OF CYCLIST INJURED=u'0', NUMBER OF CYCLIST KILLED=u'0', NUMBER OF MOTORIST INJURED=u'0', NUMBER OF MOTORIST KILLED=u'0', CONTRIBUTING FACTOR VEHICLE 1=u'Alcohol Involvement', CONTRIBUTING FACTOR VEHICLE 2=None, CONTRIBUTING FACTOR VEHICLE 3=None, CONTRIBUTING FACTOR VEHICLE 4=None, CONTRIBUTING FACTOR VEHICLE 5=None, UNIQUE KEY=u'3182813', VEHICLE TYPE CODE 1=u'PASSENGER VEHICLE', VEHICLE TYPE CODE 2=None, VEHICLE TYPE CODE 3=None, VEHICLE TYPE CODE 4=None, VEHICLE TYPE CODE 5=None),
 Row(DATE=u'03/08/2015', TIME=u'0:01', BOROUGH=u'MANHATTAN', ZIP CODE=u'100

In [11]:
sqlContext.sql('select percentile(tripduration, 0.5) from citibike').collect()

[Row(percentile(tripduration, CAST(0.5 AS DOUBLE))=529.0)]

In [12]:
# Wednesday April 26, 2017 STARTS HERE
# Files required: citibike.csv, DOE_High_School_Directory_2014-2015.csv, SAT_Results.csv, twitter_1k.jsonl, subway.zip

In [13]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [15]:
dfSchools = spark.read.load(HSD_FN, format='csv', header=True, inferSchema=True).na.drop(subset=['boro'])
dfSchools = dfSchools.filter(dfSchools['total_students']>500)
dfSchools = dfSchools.select('dbn', 'boro')
dfSchools.count()

176

In [16]:
dfSchools.show()

+------+---------+
|   dbn|     boro|
+------+---------+
|01M450|Manhattan|
|01M539|Manhattan|
|01M696|Manhattan|
|02M374|Manhattan|
|02M400|Manhattan|
|02M408|Manhattan|
|02M412|Manhattan|
|02M413|Manhattan|
|02M416|Manhattan|
|02M418|Manhattan|
|02M420|Manhattan|
|02M425|Manhattan|
|02M475|Manhattan|
|02M489|Manhattan|
|02M519|Manhattan|
|02M520|Manhattan|
|02M529|Manhattan|
|02M542|Manhattan|
|02M580|Manhattan|
|02M600|Manhattan|
+------+---------+
only showing top 20 rows



In [20]:
dfScores = sqlContext.read.load(SAT_FN, format='csv', header=True, inferSchema=True)
dfScores.select(dfScores['`SAT Math Avg. Score`'].cast('int'))

DataFrame[SAT Math Avg. Score: int]

In [23]:
dfScores.dtypes

[('DBN', 'string'),
 ('SCHOOL NAME', 'string'),
 ('Num of SAT Test Takers', 'string'),
 ('SAT Critical Reading Avg. Score', 'string'),
 ('SAT Math Avg. Score', 'string'),
 ('SAT Writing Avg. Score', 'string')]

In [24]:
dfMScores = dfScores.select('DBN', 
                            dfScores['`SAT Math Avg. Score`'].cast('int').alias('score'), # alias to name the column
                            dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers'),).na.drop() 
# the drop is required because there are some cells which are empty or cannot be casted to int.
dfMScores = dfMScores.select('DBN', 
                             (dfMScores.score*dfMScores.ntakers).alias('sum_scores'), 
                             'ntakers')
dfMScores.show()

+------+----------+-------+
|   DBN|sum_scores|ntakers|
+------+----------+-------+
|02M047|      6400|     16|
|21K410|    207575|    475|
|30Q301|     43120|     98|
|17K382|     22066|     59|
|18K637|     13335|     35|
|32K403|     18300|     50|
|09X365|     18306|     54|
|11X270|     22064|     56|
|05M367|     12078|     33|
|14K404|     24276|     68|
|30Q575|     66420|    135|
|13K336|      3366|      9|
|04M635|     17712|     48|
|24Q264|     40406|     89|
|17K408|     19494|     57|
|19K618|     22260|     60|
|27Q309|     13644|     36|
|32K552|     24388|     67|
|13K499|     26208|     72|
|07X600|     30400|     76|
+------+----------+-------+
only showing top 20 rows



In [31]:
df = dfSchools.join(dfMScores, dfSchools.dbn==dfMScores.DBN, how='inner')
df = df.groupBy('boro').sum('sum_scores', 'ntakers')
df.show()

+-------------+---------------+------------+
|         boro|sum(sum_scores)|sum(ntakers)|
+-------------+---------------+------------+
|       Queens|        5190534|       10942|
|     Brooklyn|        4544126|        9322|
|Staten Island|        1406967|        2944|
|    Manhattan|        3206992|        6228|
|        Bronx|        1619364|        3444|
+-------------+---------------+------------+



In [36]:
df.withColumn('avg', (df[1] / df[2]).cast('int')).select('boro', 'avg').show()

+-------------+---+
|         boro|avg|
+-------------+---+
|       Queens|474|
|     Brooklyn|487|
|Staten Island|477|
|    Manhattan|514|
|        Bronx|470|
+-------------+---+



In [37]:
# Everything above this cell was the required part for the lab. 

In [40]:
dfTweets = sqlContext.read.load('twitter_1k.jsonl', format='json')

In [41]:
dfTweets.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true

In [49]:
dfTweets = sqlContext.read.load('twitter_1k.jsonl', format='json')
hashtags = dfTweets.select('entities.hashtags.text')
hashtags = hashtags.filter(sf.size('text') > 0)
hashtags = hashtags.select(sf.explode('text').alias('hashtag'))
hashtags = hashtags.groupBy('hashtag').count().orderBy('count', ascending=False)
hashtags.show()

+------------------+-----+
|           hashtag|count|
+------------------+-----+
|USSaluteouttuesday|   33|
|               NYC|    5|
| privatepassSports|    4|
|               nyc|    4|
|         redcarpet|    3|
|         superbowl|    3|
|               Job|    3|
|   SuperBowlXLVIII|    3|
|              espn|    3|
|       RobinThicke|    3|
|    henleyvaporium|    2|
|             ncaaw|    2|
|      ESPNTHEPARTY|    2|
|             facts|    2|
|         SuperBowl|    2|
|           NewYork|    2|
|              soho|    2|
|                IT|    2|
|               Raw|    2|
|              Jobs|    2|
+------------------+-----+
only showing top 20 rows



In [ ]:
# TODO python run script to be uploaded to Blackboard